In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import roc_auc_score, roc_curve
from nltk.stem.snowball import EnglishStemmer

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
pd.set_option("display.max_colwidth", 500)

### Load the data

In [2]:
ctl_df = pd.read_csv('cocktail_info.csv', sep=',')

In [3]:
ctl_df.head()

,Name,Description,Recipe,Ingredients,Nutrition Facts,Video Link
0,American Beauty Cocktail,"Unfortunately, we have no description for this...",Woops... We couldn't retrieve the exact recipe...,| 1 oz: Brandy | .5 oz: Dry Vermouth | .25 oz...,Calories: 185kcal | Carbohydrates: 14g | Prote...,There doesn't seem to be an instructional vide...
1,Azzuro,"Unfortunately, we have no description for this...",Shake and strain into an ice-filled collins gl...,| 1 oz: Exotic-Fruit Liqueur | 1 oz: Blue Cur...,There doesn't seem to be any nutritional facts...,There doesn't seem to be an instructional vide...
2,Apple Fairy,"Unfortunately, we have no description for this...",For this recipe. make some Juice - Apple Juice...,| 2/3 oz: Herbal Liqueur | 2/3 oz: Apple Vodk...,There doesn't seem to be any nutritional facts...,There doesn't seem to be an instructional vide...
3,Black Rose Bacardi,"Unfortunately, we have no description for this...",Shake or stir. pour it into a coctail glass. a...,|: Ice | 1 oz: White Rum | 2 oz: Cola | 2 oz:...,There doesn't seem to be any nutritional facts...,There doesn't seem to be an instructional vide...
4,Blue Latvian,"Unfortunately, we have no description for this...",Pour the Jack Daniel`s whiskey into a mixing g...,| 1 oz: Bourbon Whiskey |: Raspberry Juice |:...,There doesn't seem to be any nutritional facts...,There doesn't seem to be an instructional vide...


In [8]:
ctl_ingredients = ctl_df['Ingredients']

In [10]:
ctl_ingredients = pd.DataFrame(ctl_ingredients)
ctl_ingredients

,Ingredients
0,| 1 oz: Brandy | .5 oz: Dry Vermouth | .25 oz...
1,| 1 oz: Exotic-Fruit Liqueur | 1 oz: Blue Cur...
2,| 2/3 oz: Herbal Liqueur | 2/3 oz: Apple Vodk...
3,|: Ice | 1 oz: White Rum | 2 oz: Cola | 2 oz:...
4,| 1 oz: Bourbon Whiskey |: Raspberry Juice |:...
...,...
5111,| 2 oz: Vodka | 1 tsp tsp: Bitters
5112,|: Crushed Ice | 1.5 oz: Brandy | 1 oz: Madei...
5113,| 1.5 oz: Brandy | .5 oz: Tonic Water |: Water
5114,| 1 oz: Dark Rum | 1 oz: Brandy | 1 tbsp: Lim...


## 1: remove all but the ingredient name using regex  
## 2: split ingredients into columns

#### 1: remove all but the ingredient name

In [11]:
ctl_ingredients

,Ingredients
0,| 1 oz: Brandy | .5 oz: Dry Vermouth | .25 oz...
1,| 1 oz: Exotic-Fruit Liqueur | 1 oz: Blue Cur...
2,| 2/3 oz: Herbal Liqueur | 2/3 oz: Apple Vodk...
3,|: Ice | 1 oz: White Rum | 2 oz: Cola | 2 oz:...
4,| 1 oz: Bourbon Whiskey |: Raspberry Juice |:...
...,...
5111,| 2 oz: Vodka | 1 tsp tsp: Bitters
5112,|: Crushed Ice | 1.5 oz: Brandy | 1 oz: Madei...
5113,| 1.5 oz: Brandy | .5 oz: Tonic Water |: Water
5114,| 1 oz: Dark Rum | 1 oz: Brandy | 1 tbsp: Lim...


In [ ]:
for i in ctl_ingredients['Ingredients']:
    i.replace(everything from "|" to the next uppercase, by ' ')

In [19]:
ctl_ingredients_clean = ctl_ingredients['Ingredients'].str.extractall(r"([A-Z].*?\w+(?=[A-Z]))")

In [17]:
ctl_ingredients_clean

0
     match                            
62   0      Banana Schnapps | .5 oz: O
147  0                   L: Gin | 10 m
     1            L: Triple Sec | 20 m
     2         L: Coconut Cream | 10 m
     3        L: Banana Liqueur | 10 m
...                                ...
5029 2          L: Orange Juice | 60 m
     3       L: Cranberry Juice | 25 m
     4         L: Cherry Brandy | 60 m
     5           L: Energy Soda | 25 m
5072 0           Apple Juice | 1 oz: O

[186 rows x 1 columns]